In [14]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import time
from skfuzzy import cmeans
import artificial_bee_colony.colonies as cols
import artificial_bee_colony.goals as goals
import abc_plusplus_128dim
import abc_plusplus_64dim
from skopt import BayesSearchCV
import sklearn
from skopt.space import Real, Categorical, Integer
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import metrics
import scipy
import itertools
import random
from scipy.spatial import distance

In [2]:
def load_pa(filename):
    result = []
    with open(filename) as f:
        for line in f:
            if line[:5] == '-----':
                break
        for line in f:
            result.append(int(line))
    return result

def map_labels(original, generated):
    org_values = list(set(original))
    org_i_mapping = {org_values[i]:i for i in range(len(org_values))}

    gen_values = list(set(generated))
    gen_i_mapping = {gen_values[i]:i for i in range(len(gen_values))}

    cost_matrix = np.zeros([len(org_values), len(gen_values)])
    for i in range(len(org_values)):
        indices = np.where(original == org_values[i])[0]
        counts = Counter(generated[indices])
        for k in counts:
            cost_matrix[i][gen_i_mapping[k]] = -counts[k]
    
    org_i, gen_i = scipy.optimize.linear_sum_assignment(cost_matrix)
    gen_i = gen_i[org_i.argsort()]
    org_i = np.sort(org_i)
    
    r = []
    for g in generated:
        for i, j in zip(gen_i, org_i):
            if g == gen_values[i]:
                r.append(org_values[j])
                
    return r

def map_fuzzy_labels(original_f, generated_f, padding=True):
    original = original_f.argmax(axis=0)
    generated = generated_f.argmax(axis=0)
    
    org_values = list(set(original))
    org_i_mapping = {org_values[i]:i for i in range(len(org_values))}

    gen_values = list(set(generated))
    gen_i_mapping = {gen_values[i]:i for i in range(len(gen_values))}

    cost_matrix = np.zeros([len(org_values), len(gen_values)])
    for i in range(len(org_values)):
        indices = np.where(original == org_values[i])[0]
        counts = Counter(generated[indices])
        for k in counts:
            cost_matrix[i][gen_i_mapping[k]] = -counts[k]
    
    org_i, gen_i = scipy.optimize.linear_sum_assignment(cost_matrix)
    gen_i = gen_i[org_i.argsort()]
    org_i = np.sort(org_i)
    
    r = []
    for i in gen_i:
        r.append(generated_f[org_i[i]])
        
    if len(org_values) > len(gen_values):
        for i in range(len(org_values) - len(gen_values)):
            r.append(np.zeros(generated_f.shape[1]))
                
    return np.array(r)

def compute_cost(weights, data):
    result = 0
    for cluster in weights:
        center = sum((data.T * cluster).T) / sum(cluster)
        for vector, weight in zip(data, cluster):
            result += weight * distance.euclidean(vector, center)
    return result

In [3]:
dimset_128 = np.loadtxt('data/dim_set/dim128.txt')
dimset_128_partitions = load_pa('data/dim_set/dim128.pa.txt')
dimset_128_fuzzy_gt = np.load('data/dim_set/dim128-fuzzy_gt.npy')

# Hyper-parameter selection

In [4]:
sample = np.array(random.sample(range(len(dimset_128)), k=258))

In [12]:
sample

array([ 326,   58,   10,  210,  517,  264, 1001,  204,  332,  894,  616,
        526,  758,  548,   97,  440,  698,  438,  924,  721,  716,  684,
        159,  584,  644,  661,  469,  265,  511,  572,  311,  792,  173,
        656,  606,  415,  729,   21,   85,  532,  573,  845,  460,  983,
         51,   56,  920,  269,  103,  467, 1004,  903,  599,  663,  263,
        519,  586,  211,  447,  622,  551,  915,  744,  867, 1002,  872,
       1018,  431,  892,  107,   49,   31,  221,  933,  723,  244,  725,
        396,   81,  328,  143,  897,  726,  574,  890,  377,  385,  675,
        418,  563,  695,  222,  386,  596,  361,  627,  544,  368,  543,
        687,  623,  938,  921,  557,  546,  769,  250,  806,  805,  270,
        885,  889,  961,  126,  378,  705,  445,  240,  884,  112,  251,
         65,  142, 1013,  626,  967,  505,  437,  243,   43,  570, 1011,
        771,  303,  746,  646,  972,  369,  620,  395,  503,  156,  100,
        459,  151,  597,  174,   52,  278,  207, 10

In [14]:
sample_data = dimset_128[sample]
sample_labels = np.array(dimset_128_partitions)[sample]

In [6]:
Counter(sample_labels).most_common()

[(5, 20),
 (10, 19),
 (7, 19),
 (14, 18),
 (15, 18),
 (6, 17),
 (4, 17),
 (9, 17),
 (13, 16),
 (12, 15),
 (2, 15),
 (1, 14),
 (16, 14),
 (11, 13),
 (3, 13),
 (8, 13)]

In [22]:
def create_estimator(data):
    class ABCEstimator(sklearn.base.BaseEstimator):
        def __init__(self, pop=20, cycles=1000, limit=200, n_clusters=16):
            self.pop = pop
            self.cycles = cycles
            self.limit = limit
            self.n_clusters = n_clusters

        def fit(self, data, y=None):
            self._colony = abc_plusplus_128dim.ArtificialBeeColony(data, self.n_clusters, int(self.pop), int(self.limit))
            self._colony.fit(int(self.cycles))
            return self

        def score(self, data):
            return self._colony.score()
    return ABCEstimator

def create_tournament_estimator(data):
    class TournamentABCEstimator(sklearn.base.BaseEstimator):
        def __init__(self, pop=20, cycles=1000, limit=200, n_clusters=16):
            self.pop = pop
            self.cycles = cycles
            self.limit = limit
            self.n_clusters = n_clusters

        def fit(self, data, y=None):
            self._colony = abc_plusplus_128dim.TournamentArtificialBeeColony(data, self.n_clusters, int(self.pop), int(self.limit))
            self._colony.fit(int(self.cycles))
            return self

        def score(self, data):
            return self._colony.score()
    return TournamentABCEstimator

In [31]:
estimator = create_estimator(sample_data)(20, 1000, 200)
bayes = BayesSearchCV(estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400)
}, n_iter=10, verbose=1)

In [32]:
bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   56.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.3min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.6min finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=ABCEstimator(cycles=1000, limit=200, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=10, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400)},
       verbose=1)

In [33]:
bayes.best_params_

{'limit': 309, 'pop': 50}

In [18]:
estimator = create_estimator(sample_data)(20, 1000, 200)
bayes = BayesSearchCV(estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400)
}, n_iter=30, verbose=1)

bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.7min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   56.1s finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=ABCEstimator(cycles=1000, limit=200, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400)},
       verbose=1)

In [20]:
bayes.best_params_

{'limit': 317, 'pop': 30}

In [23]:
estimator = create_tournament_estimator(sample_data)(20, 1000, 200)
t_bayes = BayesSearchCV(estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400)
}, n_iter=30, verbose=1)

t_bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.3min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   56.1s finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   55.9s finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=TournamentABCEstimator(cycles=1000, limit=200, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400)},
       verbose=1)

In [24]:
t_bayes.best_params_

{'limit': 25, 'pop': 50}

In [26]:
def create_mod_estimator(data):
    class ModABCEstimator(sklearn.base.BaseEstimator):
        def __init__(self, pop=20, cycles=1000, limit=200, f=0.8, mr=0.1, n_clusters=16):
            self.pop = pop
            self.cycles = cycles
            self.limit = limit
            self.f = f
            self.mr = mr
            self.n_clusters = n_clusters

        def fit(self, data, y=None):
            self._colony = abc_plusplus_128dim.ModArtificialBeeColony(data, self.n_clusters, int(self.pop), int(self.limit), float(self.f), float(self.mr))
            self._colony.fit(int(self.cycles))
            return self

        def score(self, data):
            return self._colony.score()
    return ModABCEstimator

def create_tournament_mod_estimator(data):
    class TournamentModABCEstimator(sklearn.base.BaseEstimator):
        def __init__(self, pop=20, cycles=1000, limit=200, f=0.8, mr=0.1, n_clusters=16):
            self.pop = pop
            self.cycles = cycles
            self.limit = limit
            self.f = f
            self.mr = mr
            self.n_clusters = n_clusters

        def fit(self, data, y=None):
            self._colony = abc_plusplus_128dim.TournamentModArtificialBeeColony(data, self.n_clusters, int(self.pop), int(self.limit), float(self.f), float(self.mr))
            self._colony.fit(int(self.cycles))
            return self

        def score(self, data):
            return self._colony.score()
    return TournamentModABCEstimator

In [9]:
mod_estimator = create_mod_estimator(sample_data)(20, 1000, 200, 0.1, 0.7, 16)
mod_bayes = BayesSearchCV(mod_estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400),
    'f': Real(0, 1),
    'mr': Real(0, 1),
}, n_iter=15, verbose=1)

In [10]:
mod_bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.7min finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=ModABCEstimator(cycles=1000, f=0.1, limit=200, mr=0.7, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=15, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400), 'f': Real(low=0, high=1, prior='uniform', transform='identity'), 'mr': Real(low=0, high=1, prior='uniform', transform='identity')},
       verbose=1)

In [11]:
mod_bayes.best_params_

{'f': 0.4891644921988578, 'limit': 95, 'mr': 0.11924786590959738, 'pop': 23}

In [15]:
mod_estimator = create_mod_estimator(sample_data)(20, 1000, 200, 0.1, 0.7, 16)
mod_bayes = BayesSearchCV(mod_estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400),
    'f': Real(0, 1),
    'mr': Real(0, 1),
}, n_iter=30, verbose=1)

mod_bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.5min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.2min finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=ModABCEstimator(cycles=1000, f=0.1, limit=200, mr=0.7, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400), 'f': Real(low=0, high=1, prior='uniform', transform='identity'), 'mr': Real(low=0, high=1, prior='uniform', transform='identity')},
       verbose=1)

In [16]:
mod_bayes.best_params_

{'f': 0.41230542901888145, 'limit': 187, 'mr': 0.18862883068061148, 'pop': 41}

In [27]:
mod_estimator = create_tournament_mod_estimator(sample_data)(20, 1000, 200, 0.1, 0.7, 16)
mod_t_bayes = BayesSearchCV(mod_estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400),
    'f': Real(0, 1),
    'mr': Real(0, 1),
}, n_iter=30, verbose=1)

mod_t_bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.6min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1min finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=TournamentModABCEstimator(cycles=1000, f=0.1, limit=200, mr=0.7,
             n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400), 'f': Real(low=0, high=1, prior='uniform', transform='identity'), 'mr': Real(low=0, high=1, prior='uniform', transform='identity')},
       verbose=1)

In [28]:
mod_t_bayes.best_params_

{'f': 0.9995027160733371, 'limit': 393, 'mr': 0.00852256385047623, 'pop': 49}

# Dim128 dataset evaluation

In [109]:
start = time.process_time()
_, u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 20_000)
print(time.process_time() - start)

51.15625


In [110]:
Counter(u.argmax(axis=0))

Counter({0: 1024})

In [165]:
u

array([[0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       ...,
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625]])

In [111]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), u.argmax(axis=0)))

0.0625

In [146]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, u))

0.062255859374999986

In [164]:
compute_cost(u, dimset_128)

601614.9660454898

In [149]:
start = time.process_time()
_, u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 100_000)
print(time.process_time() - start)

256.609375


In [152]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), u.argmax(axis=0)))

0.0625

In [153]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, u))

0.062255859374999986

In [161]:
compute_cost(u, dimset_128)

601614.9660454898

In [151]:
Counter(u.argmax(axis=0))

Counter({0: 1024})

In [39]:
colony = abc_plusplus_128dim.ArtificialBeeColony(dimset_128, 16, 30, 317)
start = time.process_time()
cpp_result = np.array(colony.optimize(20_000))
print(time.process_time() - start)

14202.171875


In [45]:
#np.save('results/ABC_experiment1.npy', cpp_result)

In [41]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), cpp_result.argmax(axis=0)))

0.5302734375

In [101]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, cpp_result))

0.04194288250371845

In [160]:
compute_cost(cpp_result, dimset_128)

543142.3306296628

In [40]:
Counter(cpp_result.argmax(axis=0))

Counter({14: 65,
         0: 62,
         2: 63,
         5: 51,
         10: 69,
         11: 72,
         7: 90,
         3: 65,
         1: 63,
         13: 55,
         15: 57,
         12: 94,
         8: 58,
         9: 64,
         6: 46,
         4: 50})

In [166]:
start = time.process_time()
_, b_u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 1000, init=cpp_result)
print(time.process_time() - start)

2.5625


In [168]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), b_u.argmax(axis=0)))

1.0

In [169]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, b_u))

7.875039027644074e-06

In [170]:
compute_cost(b_u, dimset_128)

18465.50909884966

In [167]:
Counter(b_u.argmax(axis=0))

Counter({2: 64,
         7: 64,
         0: 64,
         11: 64,
         3: 64,
         4: 64,
         10: 64,
         8: 64,
         12: 64,
         6: 64,
         9: 64,
         1: 64,
         15: 64,
         13: 64,
         14: 64,
         5: 64})

In [4]:
mod_colony = abc_plusplus_128dim.ModArtificialBeeColony(dimset_128, 16, 41, 187, 0.41230542901888145, 0.18862883068061148)
start = time.process_time()
mod_cpp_result = np.array(mod_colony.optimize(20_000))
print(time.process_time() - start)

20780.640625


In [5]:
#np.save('results/ABC_DE_experiment1.npy', mod_cpp_result)

In [7]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), mod_cpp_result.argmax(axis=0)))

0.583984375

In [8]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, mod_cpp_result))

0.05186391477070981

In [9]:
compute_cost(mod_cpp_result, dimset_128)

336926.0519133177

In [6]:
Counter(mod_cpp_result.argmax(axis=0))

Counter({5: 53,
         10: 103,
         2: 117,
         6: 86,
         1: 103,
         7: 85,
         9: 44,
         13: 56,
         12: 66,
         14: 35,
         11: 51,
         0: 51,
         4: 51,
         15: 42,
         8: 36,
         3: 45})

In [10]:
start = time.process_time()
_, mb_u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 1000, init=mod_cpp_result)
print(time.process_time() - start)

3.359375


In [11]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), mb_u.argmax(axis=0)))

1.0

In [12]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, mb_u))

7.87503902764407e-06

In [13]:
compute_cost(mb_u, dimset_128)

18465.509098849758

In [18]:
ts_colony = abc_plusplus_128dim.TournamentArtificialBeeColony(dimset_128, 16, 50, 25)
start = time.process_time()
ts_cpp_result = np.array(ts_colony.optimize(20_000))
print(time.process_time() - start)

24980.84375


In [24]:
#np.save('results/ABC_TS_experiment1.npy', ts_cpp_result)

In [25]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), ts_cpp_result.argmax(axis=0)))

0.9033203125

In [26]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, ts_cpp_result))

0.01217944457710815

In [27]:
compute_cost(ts_cpp_result, dimset_128)

89716.69404415038

In [61]:
start = time.process_time()
_, tb_u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 1000, init=ts_cpp_result)
print(time.process_time() - start)

4.609375


In [62]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), tb_u.argmax(axis=0)))

1.0

In [63]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, tb_u))

7.875039027644065e-06

In [64]:
compute_cost(tb_u, dimset_128)

18465.509098849732

In [29]:
ts_mod_colony = abc_plusplus_128dim.TournamentModArtificialBeeColony(dimset_128, 16, 49, 393, 0.9995027160733371, 0.00852256385047623)
start = time.process_time()
ts_mod_cpp_result = np.array(ts_mod_colony.optimize(20_000))
print(time.process_time() - start)

24760.78125


In [30]:
#np.save('results/ABC_DE_TS_experiment1.npy', ts_mod_cpp_result)

In [32]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), ts_mod_cpp_result.argmax(axis=0)))

0.923828125

In [33]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, ts_mod_cpp_result))

0.00879608578103245

In [34]:
compute_cost(ts_mod_cpp_result, dimset_128)

182765.90647453684

In [65]:
start = time.process_time()
_, tmb_u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 1000, init=ts_mod_cpp_result)
print(time.process_time() - start)

5.546875


In [66]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), tmb_u.argmax(axis=0)))

1.0

In [67]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, tmb_u))

7.87503902764407e-06

In [68]:
compute_cost(tmb_u, dimset_128)

18465.509098849696

# Worms dataset evaluation

In [20]:
worms = np.loadtxt('data/worms/worms_64d_sample.txt')
worms_partitions = load_pa('data/worms/worms_64d_sample-gt.pa')
worms_fuzzy_partitions = np.load('data/worms/worms_64d_sample-fuzzy_gt.npy')

In [52]:
start = time.process_time()
_, u, _, _, _, _, _ = cmeans(worms.T, 25, 2, 0.0, 20_000)
print(time.process_time() - start)

135.90625


In [53]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), u.argmax(axis=0)))

0.23095703125

In [54]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, u))

0.03410390625000001

In [55]:
compute_cost(u, worms)

21516608.42291214

In [56]:
start = time.process_time()
_, u, _, _, _, _, _ = cmeans(worms.T, 25, 2, 0.0, 100_000)
print(time.process_time() - start)

710.75


In [57]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), u.argmax(axis=0)))

0.23486328125

In [58]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, u))

0.03410390625000001

In [59]:
compute_cost(u, worms)

21516608.42291214

In [16]:
colony = abc_plusplus_64dim.ArtificialBeeColony(worms, 25, 30, 317)
start = time.process_time()
cpp_result = np.array(colony.optimize(20_000))
print(time.process_time() - start)

21232.046875


In [17]:
#np.save('results/ABC_experiment2.npy', cpp_result)

In [35]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), cpp_result.argmax(axis=0)))

0.1376953125

In [22]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, cpp_result))

0.03426014510849611

In [23]:
compute_cost(cpp_result, worms)

21433288.55355315

In [46]:
start = time.process_time()
_, b_u, _, _, _, _, _ = cmeans(worms.T, 25, 2, 0.0, 1000, init=cpp_result)
print(time.process_time() - start)

6.640625


In [47]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), b_u.argmax(axis=0)))

0.20947265625

In [48]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, b_u))

0.03410390625000001

In [49]:
compute_cost(b_u, worms)

21516608.42291214

In [69]:
colony = abc_plusplus_64dim.ModArtificialBeeColony(worms, 25, 41, 187, 0.41230542901888145, 0.18862883068061148)
start = time.process_time()
cpp_result = np.array(colony.optimize(20_000))
print(time.process_time() - start)

32252.046875


In [70]:
#np.save('results/ABC_DE_experiment2.npy', cpp_result)

In [73]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), cpp_result.argmax(axis=0)))

0.2919921875

In [72]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, cpp_result))

0.05080897990547265

In [71]:
compute_cost(cpp_result, worms)

20441423.64527559

In [79]:
start = time.process_time()
_, mb_u, _, _, _, _, _ = cmeans(worms.T, 25, 2, 0.0, 1000, init=cpp_result)
print(time.process_time() - start)

5.3125


In [83]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), mb_u.argmax(axis=0)))

0.26171875

In [81]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, mb_u))

0.03410390625000001

In [82]:
compute_cost(mb_u, worms)

21516608.42291214

In [85]:
colony = abc_plusplus_64dim.TournamentArtificialBeeColony(worms, 25, 50, 25)
start = time.process_time()
cpp_result = np.array(colony.optimize(20_000))
print(time.process_time() - start)

37920.703125


In [87]:
#np.save('results/ABC_TS_experiment2.npy', cpp_result)

In [88]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), cpp_result.argmax(axis=0)))

0.3544921875

In [89]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, cpp_result))

0.03683779463052084

In [90]:
compute_cost(cpp_result, worms)

20442686.855450775

In [91]:
start = time.process_time()
_, tb_u, _, _, _, _, _ = cmeans(worms.T, 25, 2, 0.0, 1000, init=cpp_result)
print(time.process_time() - start)

5.171875


In [92]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), tb_u.argmax(axis=0)))

0.26220703125

In [93]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, tb_u))

0.03410390625000001

In [94]:
compute_cost(tb_u, worms)

21516608.42291214

In [86]:
colony = abc_plusplus_64dim.TournamentModArtificialBeeColony(worms, 25, 49, 393, 0.9995027160733371, 0.00852256385047623)
start = time.process_time()
cpp_result = np.array(colony.optimize(20_000))
print(time.process_time() - start)

37614.875


In [95]:
#np.save('results/ABC_TS_DE_experiment2.npy', cpp_result)

In [97]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), cpp_result.argmax(axis=0)))

0.51708984375

In [98]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, cpp_result))

0.024096567410752216

In [99]:
compute_cost(cpp_result, worms)

20310359.852507662

In [100]:
start = time.process_time()
_, tmb_u, _, _, _, _, _ = cmeans(worms.T, 25, 2, 0.0, 1000, init=cpp_result)
print(time.process_time() - start)

5.140625


In [101]:
metrics.accuracy_score(worms_partitions, map_labels(np.array(worms_partitions), tmb_u.argmax(axis=0)))

0.22705078125

In [102]:
metrics.mean_squared_error(worms_fuzzy_partitions.T, map_fuzzy_labels(worms_fuzzy_partitions.T, tmb_u))

0.03410390625000001

In [103]:
compute_cost(tmb_u, worms)

21516608.42291214

In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#